## Dgraph Migrate + Live Loader tutorial

In [1]:
!docker pull dgraph/standalone

Using default tag: latest
latest: Pulling from dgraph/standalone

dc954fb5: Pulling fs layer 
b700ef54: Pulling fs layer 
3d98f3d8: Pulling fs layer 
0af53e79: Pulling fs layer 
acacb54b: Pulling fs layer 
5d8d161a: Pulling fs layer 
Digest: sha256:840b8125bd22c4519f3cfc0ffebed6ae8b027d698f945a8ff7a0377cf89eae63
Status: Downloaded newer image for dgraph/standalone:latest
docker.io/dgraph/standalone:latest


We have pulled the dgraph/standalone docker image. This next code block prints the imageID.

In [2]:
imageSearchResult = !docker images | grep dgraph/standalone | grep latest | awk '{print $3}'
imageID = imageSearchResult[0]
imageID

'ac77647e1c49'

We will run the docker container in the background.

In [6]:
%%script bash --bg --out /tmp/output.txt

docker run dgraph/standalone -p 8080:8080

The docker container ID will be used in every step of this tutorial. We save our docker container ID as a variable containerID for easy reference.

In [5]:
containerSearchResult = !docker ps | grep dgraph/standalone | awk '{print $1}'
containerID = containerSearchResult[0]
containerID

IndexError: list index out of range

Before installing MySQL inside the docker container, we must update the apt package manager. Call Apt-get update from within the docker container as shown below.

In [20]:
!docker exec -it $containerID apt-get update

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1091 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3420 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2925 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1391 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/

We set the front end to be noninteractive to preserve the Jupyter notebook format. Use apt-get to install the mysql-server inside our docker container.

In [25]:
!docker exec -it $containerID bash -c "DEBIAN_FRONTEND=noninteractive apt-get -y install mysql-server"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mysql-server is already the newest version (8.0.34-0ubuntu0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


Run mysqld in the background. Output prints mysql instance running so it is clear for the user.

In [26]:
!docker exec -it $containerID bash -c 'mysqld &'
!docker exec -it $containerID bash -c 'ps auwx | grep mysqld | grep -v grep'

mysql     1058  2.1  4.9 2381512 396168 pts/0  Sl   17:56   0:02 mysqld


Now we can make mysql commands from within the docker container. Before adding data, we need to create a database. Here our example database is called testdb.

In [27]:
!docker exec -it $containerID mysql -u root -e 'create database testdb'

Set a password for the mysql databse with mysql_native_password.

In [28]:
!docker exec -it $containerID mysql -u root -e "alter user root@localhost identified with mysql_native_password by 'password';"

Make a file with our login credentials. For our example we set password=password.

In [29]:
!docker exec -it $containerID rm -f /tmp/login.cnf 
!docker exec -it $containerID bash -c "echo '[client]' >> /tmp/login.cnf" 
!docker exec -it $containerID bash -c "echo 'password=password' >> /tmp/login.cnf" 

Now we can create a table in the mysql database. For this example we've created a table called Person which includes two columns, Name and Age.

In [30]:
!docker exec -it $containerID mysql --defaults-extra-file=/tmp/login.cnf testdb -e 'create table Roster (Name varchar(50), Jersey int);'

Add a few rows of data into the database. This data will eventually be converted into N-quad entries for loading into Dgraph.

In [33]:
!docker exec -it $containerID mysql --defaults-extra-file=/tmp/login.cnf testdb -e 'insert into Roster values ("LeBron",23);'
!docker exec -it $containerID mysql --defaults-extra-file=/tmp/login.cnf testdb -e 'insert into Roster values ("Steph",30);'
!docker exec -it $containerID mysql --defaults-extra-file=/tmp/login.cnf testdb -e 'insert into Roster values ("Kobe",24);'

In [172]:
!docker exec -it $containerID bash -c "cd /tmp ; pwd"

/tmp


Now that we have a running mysql database running inside the docker container, we can use the migrate tool to generate our schema and output files which will then be loaded into Dgraph. For more details on dgraph migrate, visit this link: https://dgraph.io/docs/migration/migrate-tool/

In [78]:
!docker exec -it $containerID bash -c "cd /tmp ; rm -f schema.txt sql.rdf ; dgraph migrate --db testdb --user root --password password"

Dumping table Roster
Dumping table constraints Roster


Change the schema.txt file so that we have indexes.

In [89]:
!docker exec -it $containerID bash -c "cd /tmp; cat schema.txt | sed 's/int/int/' | sed 's/string/string @index(exact)/' > tmp.txt ; mv tmp.txt schema.txt"
!docker exec -it $containerID cat /tmp/schema.txt

Roster.Jersey: int .
Roster.Name: string @index(exact) @index(exact) .


Delete all data in Dgraph so that we don't get duplicates when we re-run this notebook.

In [85]:
print (containerID)
!docker exec -it 26861ab36825 curl localhost:8080/alter -X POST -H 'Content-Type: application/dql' -d '{"drop_all":true}'

26861ab36825
{"data":{"code":"Success","message":"Done"}}

In [87]:
!docker exec -it $containerID bash -c "cd /tmp ; dgraph live --files sql.rdf --schema schema.txt --format=rdf"

I0818 19:39:41.302428    1928 init.go:85] 

Dgraph version   : v23.0.1
Dgraph codename  : dgraph
Dgraph SHA-256   : 1d9145cf378b4e97b5f6cefd55069973c65c36f07b8e05a24b68b8ff5c5e74a4
Commit SHA-1     : 3de01e4
Commit timestamp : 2023-07-03 14:35:26 +0530
Branch           : HEAD
Go version       : go1.19.10
jemalloc enabled : true

For Dgraph official documentation, visit https://dgraph.io/docs.
For discussions about Dgraph     , visit https://discuss.dgraph.io.
For fully-managed Dgraph Cloud   , visit https://dgraph.io/cloud.

Licensed variously under the Apache Public License 2.0 and Dgraph Community License.
Copyright 2015-2023 Dgraph Labs, Inc.



Running transaction with dgraph endpoint: 127.0.0.1:9080

Processing schema file "schema.txt"
Processed schema file "schema.txt"

Found 1 data file(s) to process
Processing data file "sql.rdf"
Number of TXs run            : 1                                                                    
Number of N-Quads processed  : 18
Time spent     

Now run some queries.

In [88]:
!docker exec -it 26861ab36825 bash -c "curl localhost:8080/query -X POST -H 'Content-Type: application/dql' -d '{boo(func: eq(Roster.Name, "Steph")){Roster.Name}}' | jq"




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   361  100   313  100    48  34777   5333 --:--:-- --:--:-- --:--:-- 40111
{
  "data": {
    "boo": [
      {
        "Roster.Name": "Steph"
      },
      {
        "Roster.Name": "Steph"
      },
      {
        "Roster.Name": "Steph"
      }
    ]
  },
  "extensions": {
    "server_latency": {
      "parsing_ns": 90039,
      "processing_ns": 1095934,
      "encoding_ns": 1196024,
      "assign_timestamp_ns": 1273607,
      "total_ns": 3830530
    },
    "txn": {
      "start_ts": 147
    },
    "metrics": {
      "num_uids": {
        "Roster.Name": 3,
        "_total": 3
      }
    }
  }
}
